# 导入所需模块

In [1]:
import pandas as pd

# 数据导入

In [3]:
orders = pd.read_excel('超市数据集.xlsx', 
                       sheet_name= '订单表')
customers = pd.read_excel('超市数据集.xlsx', 
                          sheet_name= '客户表')
products = pd.read_excel('超市数据集.xlsx', 
                         sheet_name= '产品表')

In [4]:
orders.head()

,行 Id,订单 Id,订单日期,客户 Id,产品 Id,数量,销售额
0,1,US-2021-1357144,2021-04-27,14485,10002717,2,130
1,2,CN-2021-1973789,2021-06-15,10165,10004832,2,125
2,3,CN-2021-1973789,2021-06-15,10165,10001505,2,32
3,4,US-2021-3017568,2021-12-09,17170,10003746,4,321
4,5,CN-2020-2975416,2020-05-31,15730,10003452,3,1376


In [5]:
customers.head()

,客户 Id,客户名称,客户类型
0,10015,陈嫒,消费者
1,10030,贾嫒,公司
2,10045,巩嫒,公司
3,10060,罗嫒,小型企业
4,10075,余嫒,公司


In [382]:
products.head()

,物料号,类别,细分,品牌,产品名称,规格
0,10004988,办公用,美术,Stanley,铅笔刀,混合尺寸
1,10004984,办公用,信封,Jiffy,邮寄品,红色
2,10004982,办公用,标签,Harbour Creations,有色标签,白色
3,10004976,办公用,标签,Hon,可去除的标签,红色
4,10004975,办公用,收纳,Fellowes,文件夹,工业


# 数据清洗

In [6]:
orders['订单日期'] = orders['订单日期'].astype('str')
orders['数量'] = orders['数量'].astype('str')

## 数据类型

In [7]:
orders.dtypes

行 Id      int64
订单 Id    object
订单日期     object
客户 Id     int64
产品 Id     int64
数量       object
销售额       int64
dtype: object

In [8]:
customers.dtypes

客户 Id     int64
客户名称     object
客户类型     object
dtype: object

In [9]:
finance.dtypes

NameError: name 'finance' is not defined

### 修改数据类型

In [10]:
orders['订单日期'] = orders['订单日期'].astype('datetime64')
orders['数量'] = orders['数量'].apply(int)

In [11]:
orders.dtypes

行 Id              int64
订单 Id            object
订单日期     datetime64[ns]
客户 Id             int64
产品 Id             int64
数量                int64
销售额               int64
dtype: object

## 修改字段名

In [12]:
orders = orders.rename(columns={'订单 Id':'订单ID',
                                '客户 Id':'客户ID',
                                '产品 Id':'产品ID'})
customers = customers.rename(columns={'客户 Id':'客户ID'})

orders.columns

Index(['行 Id', '订单ID', '订单日期', '客户ID', '产品ID', '数量', '销售额'], dtype='object')

## 多表连接

In [13]:
data = orders.merge(customers, on='客户ID', how='left')
data.columns

Index(['行 Id', '订单ID', '订单日期', '客户ID', '产品ID', '数量', '销售额', '客户名称', '客户类型'], dtype='object')

In [16]:
data = data.merge(products, how='left', 
                  left_on='产品ID', right_on='物料号')

KeyError: '物料号'

## 剔除多余字段

In [15]:
data.drop(['物料号','行 Id'],
          axis=1,inplace=True)

KeyError: "['物料号'] not found in axis"

In [509]:
data.head()

,订单ID,订单日期,客户ID,产品ID,数量,销售额,客户名称,客户类型,类别,细分,品牌,产品名称,规格
0,US-2021-1357144,2021-04-27,14485,10002717,2,130,曾惠,公司,办公用,用品,Fiskars,剪刀,蓝色
1,CN-2021-1973789,2021-06-15,10165,10004832,2,125,许安,消费者,办公用,信封,Kraft,商业信封,银色
2,CN-2021-1973789,2021-06-15,10165,10004832,2,125,许安,消费者,办公用,信封,GlobeWeis,搭扣信封,红色
3,CN-2021-1973789,2021-06-15,10165,10001505,2,32,许安,消费者,办公用,装订,Cardinal,孔加固材料,回收
4,US-2021-3017568,2021-12-09,17170,10003746,4,321,宋良,公司,办公用,用品,Kleencut,开信刀,工业


## 时间函数 —— 剔除非分析范围数据

In [514]:
data= data[data['订单日期'].between('2019-01-01','2021-08-13')]

## 文本处理 —— 剔除不符合业务场景数据

In [401]:
data = data[~data['产品名称'].str.contains('测试')]

In [404]:
data['客户类型'].value_counts()

消费者     4845
公司      3003
小型企业    1632
Name: 客户类型, dtype: int64

In [354]:
data = data[data['客户类型']=='消费者']

# 

# 特征构造
## 消费指标

In [491]:
consume_df = data.groupby('客户ID').agg(累计消费金额=('销售额',sum), 
                         累计消费件数=('数量',sum),
                         累计消费次数=('订单日期', pd.Series.nunique), 
                         最近消费日期=('订单日期',max)
                        )

In [517]:
# 最近一次消费至今天数
from datetime import datetime
consume_df['休眠天数'] = datetime(2021,8,14) - consume_df['最近消费日期']

In [532]:
consume_df['休眠天数'] = consume_df['休眠天数'].map(lambda x:x.days)

In [533]:
consume_df.head()

,累计消费金额,累计消费件数,累计消费次数,最近消费日期,休眠天数,客单价,客单价区间,是否异常
客户ID,,,,,,,,
10015,10103,37,5,2020-09-03,345,2020.600000,"(115.834, 4772.3]",0
10030,27717,80,8,2021-04-30,106,3464.625000,"(115.834, 4772.3]",0
10045,20974,97,7,2021-07-16,29,2996.285714,"(115.834, 4772.3]",0
10060,50407,80,7,2021-08-13,1,7201.000000,"(4772.3, 9405.6]",0
10075,29920,71,7,2021-08-11,3,4274.285714,"(115.834, 4772.3]",0


In [534]:
consume_df['客单价'] = consume_df['累计消费金额']/consume_df['累计消费次数']

## 划分区间

In [18]:
consume_df['客单价区间']  = pd.cut(consume_df['客单价'],
                                   bins=5)

NameError: name 'consume_df' is not defined

In [536]:
consume_df['客单价区间'].value_counts()

(115.834, 4772.3]     505
(4772.3, 9405.6]      223
(9405.6, 14038.9]      34
(14038.9, 18672.2]     15
(18672.2, 23305.5]      2
Name: 客单价区间, dtype: int64

In [537]:
rfm_features = ['累计消费金额','累计消费次数','休眠天数']

# 剔除异常值
## Tukey's Test

In [538]:
def turkeys_test(fea):
    Q3 = consume_df[fea].quantile(0.75)
    Q1 = consume_df[fea].quantile(0.25)
    max_ = Q3+1.5*(Q3-Q1)
    min_ = Q1-1.5*(Q3-Q1)
    
    if min_<0:
        min_ =0
    
    return max_, min_

In [539]:
consume_df['是否异常'] = 0

for fea in rfm_features:
    max_, min_= turkeys_test(fea)
    outlet = consume_df[fea].between(min_,max_)  #bool
    consume_df.loc[~outlet,'是否异常']=1

In [540]:
consume_df = consume_df[consume_df['是否异常']==0]

In [541]:
consume_df[rfm_features].describe()

,累计消费金额,累计消费次数,休眠天数
count,710.000000,710.000000,710.000000
mean,17588.476056,4.216901,153.387324
std,12490.150433,1.688172,132.789272
min,139.000000,1.000000,1.000000
25%,7549.500000,3.000000,50.000000
50%,15228.000000,4.000000,99.000000
75%,25045.250000,5.000000,245.000000
max,53850.000000,8.000000,561.000000


# RFM建模

## 阈值计算：聚类 此处用二八法则代替

In [545]:
M_threshold = consume_df['累计消费金额'].quantile(0.8)
F_threshold=consume_df['累计消费次数'].quantile(0.8)
R_threshold = consume_df['休眠天数'].quantile(0.2)

In [546]:
consume_df['R'] = consume_df['休眠天数'].map(lambda x:1 if x<R_threshold else 0)
consume_df['F'] = consume_df['累计消费次数'].map(lambda x:1 if x>F_threshold else 0)
consume_df['M'] = consume_df['累计消费金额'].map(lambda x:1 if x>M_threshold else 0)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [551]:
consume_df[['休眠天数','累计消费次数', '累计消费金额', 'R','F','M']].head()

,休眠天数,累计消费次数,累计消费金额,R,F,M
客户ID,,,,,,
10015,345,5,10103,0,0,0
10030,106,8,27717,0,1,0
10045,29,7,20974,1,1,0
10060,1,7,50407,1,1,1
10075,3,7,29920,1,1,1


In [552]:
consume_df['RFM'] = consume_df['R'].apply(str)+'-' + consume_df['F'].apply(str)+'-'+ consume_df['M'].apply(str)

rfm_dict = {
    '1-1-1':'重要价值用户',
    '1-0-1':'重要发展用户',
    '0-1-1':'重要保持用户',
    '0-0-1':'重要挽留用户',
    '1-1-0':'一般价值用户',
    '1-0-0':'一般发展用户',
    '0-1-0':'一般保持用户',
    '0-0-0':'一般挽留用户'
}
consume_df['RFM人群'] = consume_df['RFM'].map(lambda x:rfm_dict[x])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [554]:
consume_df[[ 'R','F','M','RFM人群']].head()

,R,F,M,RFM人群
客户ID,,,,
10015,0,0,0,一般挽留用户
10030,0,1,0,一般保持用户
10045,1,1,0,一般价值用户
10060,1,1,1,重要价值用户
10075,1,1,1,重要价值用户


# 统计分析

## 人数占比

In [556]:
rfm_analysis = pd.DataFrame(consume_df['RFM人群'].value_counts()).rename(columns={'RFM人群':'人数'})
rfm_analysis['人群占比'] = (rfm_analysis['人数']/rfm_analysis['人数'].sum()).map(lambda x:'%.2f%%'%(x*100))

In [557]:
rfm_analysis

,人数,人群占比
一般挽留用户,427,60.14%
一般发展用户,94,13.24%
重要挽留用户,80,11.27%
一般保持用户,39,5.49%
重要发展用户,24,3.38%
重要保持用户,22,3.10%
重要价值用户,16,2.25%
一般价值用户,8,1.13%


## 透视表

In [558]:
pd.pivot_table(consume_df.reset_index(),    # DataFrame
        values='客户ID',    # 值
        index='RFM人群',    # 分类汇总依据
        columns='客单价区间',    # 列
        aggfunc=pd.Series.nunique,    # 聚合函数
        fill_value=0,    # 对缺失值的填充
        margins=True,    # 是否启用总计行/列
        dropna=False,    # 删除缺失
        margins_name='All'   # 总计行/列的名称
       ).sort_values(by='All',ascending=False)

客单价区间,"(115.834, 4772.3]","(4772.3, 9405.6]","(9405.6, 14038.9]","(14038.9, 18672.2]","(18672.2, 23305.5]",All
RFM人群,,,,,,
All,468,205,27,8,2,710
一般挽留用户,331,85,8,3,0,427
一般发展用户,74,19,0,0,1,94
重要挽留用户,1,59,15,5,0,80
一般保持用户,39,0,0,0,0,39
重要发展用户,0,19,4,0,1,24
重要保持用户,8,14,0,0,0,22
重要价值用户,7,9,0,0,0,16
一般价值用户,8,0,0,0,0,8


In [424]:
pivot_result = pd.pivot_table(consume_df.reset_index(),    # DataFrame
        values='客户ID',    # 值
        index='RFM人群',    # 分类汇总依据
        columns='客单价区间',    # 列
        aggfunc=pd.Series.nunique,    # 聚合函数
        fill_value=0,    # 对缺失值的填充
        margins=True,    # 是否启用总计行/列
        dropna=False,    # 删除缺失
        margins_name='All'   # 总计行/列的名称
       ).sort_values(by='All',ascending=False)

## 逆透视

In [455]:
pivot_table = pivot_result.reset_index().drop('All',axis=1)
pivot_table.columns = ['RFM人群','(124.359, 3871.2]', '(3871.2, 7599.4]', '(7599.4, 11327.6]', '(11327.6, 15055.8]', '(15055.8, 18784.0]']

In [456]:
pivot_table

,RFM人群,"(124.359, 3871.2]","(3871.2, 7599.4]","(7599.4, 11327.6]","(11327.6, 15055.8]","(15055.8, 18784.0]"
0,All,211.0,108.0,27.0,8.0,5.0
1,一般挽留用户,137.0,50.0,8.0,2.0,2.0
2,一般发展用户,33.0,13.0,NaN,NaN,1.0
3,重要挽留用户,NaN,10.0,17.0,5.0,2.0
4,一般保持用户,30.0,2.0,NaN,NaN,NaN
5,重要保持用户,2.0,20.0,NaN,NaN,NaN
6,一般价值用户,9.0,NaN,NaN,NaN,NaN
7,重要价值用户,NaN,9.0,NaN,NaN,NaN
8,重要发展用户,NaN,4.0,2.0,1.0,NaN


In [559]:
pivot_table.melt(id_vars='RFM人群',
                 value_vars=['(124.359, 3871.2]', '(3871.2, 7599.4]',
                             '(7599.4, 11327.6]', '(11327.6, 15055.8]',
                             '(15055.8, 18784.0]']).sort_values(by=['RFM人群','variable'],ascending=False).fillna(0)

,RFM人群,variable,value
21,重要挽留用户,"(7599.4, 11327.6]",17.0
12,重要挽留用户,"(3871.2, 7599.4]",10.0
39,重要挽留用户,"(15055.8, 18784.0]",2.0
3,重要挽留用户,"(124.359, 3871.2]",0.0
30,重要挽留用户,"(11327.6, 15055.8]",5.0
26,重要发展用户,"(7599.4, 11327.6]",2.0
17,重要发展用户,"(3871.2, 7599.4]",4.0
44,重要发展用户,"(15055.8, 18784.0]",0.0
8,重要发展用户,"(124.359, 3871.2]",0.0
35,重要发展用户,"(11327.6, 15055.8]",1.0
